In [25]:
from importlib import reload
import perplexity
reload(perplexity)
from perplexity import PerplexityAnalyzer
from transformers import GPT2Tokenizer
from gpt2 import GPT2LMHeadModel
import pandas as pd
from neuron_selection import NeuronSelector
from lda_neuron_selection import LdaNeuronSelector
import numpy as np

from matplotlib import pyplot as plt, rcParams
from matplotlib_inline.backend_inline import set_matplotlib_formats

rcParams["figure.figsize"] = (8, 5)
rcParams["figure.dpi"] = 100
set_matplotlib_formats("retina")
plt.style.use("seaborn")

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-xl")
model = GPT2LMHeadModel.from_pretrained("gpt2-xl")
model.eval()

perplexity_df = pd.read_csv("data/wiki.csv")

In [13]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [3]:
neuron_selector = NeuronSelector()
neurons_per_layer = neuron_selector.get_neurons_per_layer(n_neurons=100, percentile=0.8, method="correlation")

In [8]:
neuron_selector = LdaNeuronSelector(filename="output/output.npz")
neurons_per_layer = neuron_selector.get_lda_neurons_per_layer(n_layers=25, percentile=0.8, method="correlation")

AttributeError: 'LdaNeuronSelector' object has no attribute 'get_neurons_per_layer'

In [26]:
perp_analyzer = PerplexityAnalyzer(model, tokenizer, df=perplexity_df.sample(10))

In [27]:
perp_analyzer.get_average_perplexity(neurons_per_layer=neurons_per_layer, force_emotion="negative", force_with="per_neuron")

38026.860142534475

In [28]:
BATCH_SIZE = 5
n_batches = len(perplexity_df) // BATCH_SIZE + 1
batch = next(iter(np.array_split(perplexity_df, n_batches)))
sentences = batch["sentence"].values.tolist()
print(sentences)

["It tells the story of the making of Bandolier in the band's own words, including an extensive audio interview with Burke Shelley.", 'Under his conditions it produces pitchers that are almost completley black.', "Given that they're pulling out all the stops, they deserve all the acclaim that surrounded them for their first two albums.", 'It was their second EP and, like their first EP, was released only in Australia.', 'Contains a detailed monograph on Schisandra chinensis as well as highlights health benefits.']


In [ ]:
likelihood_sequence = self.process_sentence(
    sentences,
    neurons_per_layer=neurons_per_layer,
    force_emotion="negative",
    force_with="per_neuron",
)

In [10]:
data = [1,2,3]
columns = ["num"]

pd.DataFrame(columns=columns, data=data)

,num
0,1
1,2
2,3
